In [115]:
import json
import numpy as np
import pandas as pd
import unicodedata
import re
import math

In [173]:
df=pd.read_csv("Data/commune2021.csv")
communes_ = np.array(df["LIBELLE"])
_insee_communes, counts = np.unique(communes_, return_counts=True)
insee_communes = np.unique([x.lower() for x in _insee_communes])

In [175]:
_dbPedia = json.load(open("Data/dbPedia_extracted_demonyms3.json"))
dbPedia = { x.lower():v for x,v in _dbPedia.items()}
_infobox = json.load(open("Data/infobox_extracted_demonyms3.json"))
infobox = { x.lower():v for x,v in _infobox.items()}
habitants_com =  json.load(open("Data/habitants_com_extracted_demonyms4.json"))

In [186]:
merged = {}
for c in insee_communes:
    s1 = set([ x.lower() for x in dbPedia.get(c, [])])
    s2 = set([x.lower() for x in infobox.get(c, [])])
    s3 = set([x.lower() for x in habitants_com.get(c, [])])
    s = s1.union(s2).union(s3)
    if len(s)>0:
        merged[c] = list(s)
    

In [187]:
len(merged)

30156

In [188]:
with open("Data/merged.json", "w") as fp:
        json.dump(merged,fp)